In [9]:
using JLD
include("gyncycle.jl");

In [88]:
hillind = [4, 6, 10, 18, 20, 22, 26, 33, 36, 39, 43, 47, 49, 52, 55, 59, 65, 95, 98, 101, 103]
SAMPLEPARMS = deleteat!(collect(1:103), hillind)

SIGMA_RHO = 0.2
SIGMA_Y0 = 1000
SIGMA_PARMS = 1000;
persons = 1:3

parms, y0 = loadparms()
samplingnodes = [:sparms, :y0]
data = loadpfizer()
proposalvariance = diagm(abs2(vcat(parms[SAMPLEPARMS], y0) * 0.1));

In [ ]:
mc = []
for person in persons
    m, inp, ini = gyncmodel(data[person], parms, y0)
    setsamplers!(m, [AMM(samplingnodes, proposalvariance, adapt=:all)])
    
    @time push!(mc, mcmc(m, inp, ini, 1000, burnin=0, verbose = false, thin = 10))
end

while size(mc[1],1) < 1_000_000
    for person in persons
        @time mc[person] = mcmc(mc[person], 1000, verbose=false)        
        save("walks.jld", "mc" , [mc[person].value for person in persons])
    end
end

 36.918311 seconds (2.01 M allocations: 1.357 GB, 0.34% gc time)
 65.826863 seconds (2.35 M allocations: 1.445 GB, 0.21% gc time)
 51.427897 seconds (2.23 M allocations: 1.419 GB, 0.26% gc time)
 26.627117 seconds (2.05 M allocations: 1.363 GB, 0.49% gc time)
 66.585053 seconds (2.20 M allocations: 1.409 GB, 0.21% gc time)
 45.294433 seconds (2.22 M allocations: 1.408 GB, 0.30% gc time)
 30.674437 seconds (2.14 M allocations: 1.390 GB, 0.43% gc time)
 57.942963 seconds (2.16 M allocations: 1.394 GB, 0.23% gc time)
 48.696691 seconds (2.13 M allocations: 1.390 GB, 0.28% gc time)
 26.139199 seconds (2.10 M allocations: 1.385 GB, 0.51% gc time)
 60.866708 seconds (2.13 M allocations: 1.391 GB, 0.22% gc time)
 44.947812 seconds (2.18 M allocations: 1.401 GB, 0.30% gc time)
 26.544559 seconds (2.11 M allocations: 1.382 GB, 0.51% gc time)
 71.493065 seconds (2.07 M allocations: 1.369 GB, 0.18% gc time)
 46.496860 seconds (2.21 M allocations: 1.407 GB, 0.30% gc time)
 35.255457 seconds (2.11 